# Extract Heavy Commercial Vehicle Daily Trips from CSTDM
This script reads the county-to-county trip counts data from CSTDM (California Statewide Travel Demand Model)
and extract the trips of heavy commercial vehicles (HCVs).

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Description" data-toc-modified-id="Data-Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Description</a></span><ul class="toc-item"><li><span><a href="#Input-data:-CV_CountyToCounty_Flow_2040.csv" data-toc-modified-id="Input-data:-CV_CountyToCounty_Flow_2040.csv-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Input data: <code>CV_CountyToCounty_Flow_2040.csv</code></a></span></li><li><span><a href="#Output-data:-HCV_ODVol_2040.csv" data-toc-modified-id="Output-data:-HCV_ODVol_2040.csv-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Output data: <code>HCV_ODVol_2040.csv</code></a></span></li></ul></li><li><span><a href="#Scripts" data-toc-modified-id="Scripts-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Scripts</a></span><ul class="toc-item"><li><span><a href="#Set-row-index-as-O_County-and-D_County" data-toc-modified-id="Set-row-index-as-O_County-and-D_County-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Set row index as <code>O_County</code> and <code>D_County</code></a></span></li><li><span><a href="#Column-names-→-multi-index-(VEH,-TRIP,-PERIOD)" data-toc-modified-id="Column-names-→-multi-index-(VEH,-TRIP,-PERIOD)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Column names → multi-index (<code>VEH</code>, <code>TRIP</code>, <code>PERIOD</code>)</a></span></li><li><span><a href="#Stack-c2c_flow-table-(wide-form-→-long-form)" data-toc-modified-id="Stack-c2c_flow-table-(wide-form-→-long-form)-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Stack <code>c2c_flow</code> table (wide form → long form)</a></span></li><li><span><a href="#Select-HCV-flows" data-toc-modified-id="Select-HCV-flows-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Select HCV flows</a></span></li></ul></li></ul></div>

## Data Description

### Input data: `CV_CountyToCounty_Flow_2040.csv`
County-to-county daily trip counts, split by vehicle type, trip type, and period of day.

**File path**
`{LH_MODULE_ROOT}/Scenario_Baseline/input/CV_CountyToCounty_Flow_2040.csv`

**Fields**
- `O_County`: origin county
- `D_County`: destination county
- Other columns named by `[VEH]_[TRIP]_[PERIOD]`, where
    - `[VEH]` $\in$ `{LCV, MCV, HCV}` -- light-, medium-, heavy-duty commercial vehicles
    - `[TRIP]` $\in$ `{SHT, LD, EXT}` -- short (&lt; 50 miles), long-distance (&ge; 50 miles), externel (crosses California boarder) trips. 
    - `[PERIOD]` $\in$ `{AM, MID, PM, OFF}` -- morning, midday, afternoon, and off-peak. Four categories add up to daily total.
- Values in the table are numbers of trips

**Source**
California Statewide Travel Demand Model (http://www.dot.ca.gov/hq/tpp/offices/omsp/statewide_modeling/cstdm_trip_tables.html) and Caltrans staff

**Data access date**
2018-01-23

### Output data: `HCV_ODVol_2040.csv`

**File path**:
`{SCRATCH_FOLDER}/HCV_ODVol_2040.csv`

**Fields**:
- `O_County`: origin county
- `D_County`: destination county
- `volume`: number of trips, daily, from `O_County` to `D_County`

## Scripts

In [1]:
import os
from LH_MODULE_CONFIG import INPUT_FOLDER, SCRATCH_FOLDER
import pandas as pd

Active scenario: Range800km


In [2]:
c2c_flow = pd.read_csv(os.path.join(INPUT_FOLDER, 'CV_CountyToCounty_Flow_2040.csv'), dtype={'O_County': int, 'D_County': int})
c2c_flow.head()

,O_County,D_County,LCV_SHT_OFF,MCV_SHT_OFF,HCV_SHT_OFF,HCV_LD_OFF,MCV_EXT_OFF,HCV_EXT_OFF,LCV_SHT_AM,MCV_SHT_AM,...,HCV_SHT_MID,HCV_LD_MID,MCV_EXT_MID,HCV_EXT_MID,LCV_SHT_PM,MCV_SHT_PM,HCV_SHT_PM,HCV_LD_PM,MCV_EXT_PM,HCV_EXT_PM
0,1,1,26768.0,71271.0,32954.0,0.0,0,0,25278.0,21140.0,...,22077.0,0.0,0,0,31097.0,46288.0,16407.0,0.0,0,0
1,1,2,0.0,0.0,0.0,0.0,0,0,0.0,0.0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0
2,1,3,0.0,0.0,0.0,2.4,0,0,0.0,0.0,...,0.0,2.1,0,0,0.0,0.0,0.0,0.6,0,0
3,1,4,0.0,0.0,0.0,27.2,0,0,0.0,0.0,...,0.0,23.8,0,0,0.0,0.0,0.0,6.8,0,0
4,1,5,0.0,0.0,0.0,3.2,0,0,0.0,0.0,...,0.0,2.8,0,0,0.0,0.0,0.0,0.8,0,0


### Set row index as `O_County` and `D_County`

In [3]:
c2c_flow.set_index(c2c_flow.columns[:2].values.tolist(), inplace=True)

In [4]:
c2c_flow.head()

LCV_SHT_OFF  MCV_SHT_OFF  HCV_SHT_OFF  HCV_LD_OFF  \
O_County D_County                                                      
1        1             26768.0      71271.0      32954.0         0.0   
         2                 0.0          0.0          0.0         0.0   
         3                 0.0          0.0          0.0         2.4   
         4                 0.0          0.0          0.0        27.2   
         5                 0.0          0.0          0.0         3.2   

                   MCV_EXT_OFF  HCV_EXT_OFF  LCV_SHT_AM  MCV_SHT_AM  \
O_County D_County                                                     
1        1                   0            0     25278.0     21140.0   
         2                   0            0         0.0         0.0   
         3                   0            0         0.0         0.0   
         4                   0            0         0.0         0.0   
         5                   0            0         0.0         0.0   

                   HCV_SHT_AM  HCV_LD_AM     ...      HCV_SHT_MID  HCV_LD_MID  \
O_County D_County                            ...                                
1        1            10443.0        0.0     ...          22077.0         0.0   
         2                0.0        0.0     ...              0.0         0.0   
         3                0.0        0.9     ...              0.0         2.1   
         4                0.0       10.2     ...              0.0        23.8   
         5                0.0        1.2     ...              0.0         2.8   

                   MCV_EXT_MID  HCV_EXT_MID  LCV_SHT_PM  MCV_SHT_PM  \
O_County D_County                                                     
1        1                   0            0     31097.0     46288.0   
         2                   0            0         0.0         0.0   
         3                   0            0         0.0         0.0   
         4                   0            0         0.0         0.0   
         5                   0            0         0.0         0.0   

                   HCV_SHT_PM  HCV_LD_PM  MCV_EXT_PM  HCV_EXT_PM  
O_County D_County                                                 
1        1            16407.0        0.0           0           0  
         2                0.0        0.0           0           0  
         3                0.0        0.6           0           0  
         4                0.0        6.8           0           0  
         5                0.0        0.8           0           0  

[5 rows x 24 columns]

### Column names → multi-index (`VEH`, `TRIP`, `PERIOD`)

In [5]:
multi_idx = c2c_flow.columns\
                    .str.upper()\
                    .str.split('_', expand=True)
multi_idx.names = ['VEH','TRIP','PERIOD']

In [6]:
multi_idx

MultiIndex(levels=[['HCV', 'LCV', 'MCV'], ['EXT', 'LD', 'SHT'], ['AM', 'MID', 'OFF', 'PM']],
           labels=[[1, 2, 0, 0, 2, 0, 1, 2, 0, 0, 2, 0, 1, 2, 0, 0, 2, 0, 1, 2, 0, 0, 2, 0], [2, 2, 2, 1, 0, 0, 2, 2, 2, 1, 0, 0, 2, 2, 2, 1, 0, 0, 2, 2, 2, 1, 0, 0], [2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3]],
           names=['VEH', 'TRIP', 'PERIOD'])

In [7]:
c2c_flow.columns = multi_idx

In [8]:
c2c_flow.head()

VEH                    LCV      MCV      HCV       MCV HCV      LCV      MCV  \
TRIP                   SHT      SHT      SHT    LD EXT EXT      SHT      SHT   
PERIOD                 OFF      OFF      OFF   OFF OFF OFF       AM       AM   
O_County D_County                                                              
1        1         26768.0  71271.0  32954.0   0.0   0   0  25278.0  21140.0   
         2             0.0      0.0      0.0   0.0   0   0      0.0      0.0   
         3             0.0      0.0      0.0   2.4   0   0      0.0      0.0   
         4             0.0      0.0      0.0  27.2   0   0      0.0      0.0   
         5             0.0      0.0      0.0   3.2   0   0      0.0      0.0   

VEH                    HCV       ...                MCV HCV      LCV      MCV  \
TRIP                   SHT    LD ...      SHT    LD EXT EXT      SHT      SHT   
PERIOD                  AM    AM ...      MID   MID MID MID       PM       PM   
O_County D_County                ...                                            
1        1         10443.0   0.0 ...  22077.0   0.0   0   0  31097.0  46288.0   
         2             0.0   0.0 ...      0.0   0.0   0   0      0.0      0.0   
         3             0.0   0.9 ...      0.0   2.1   0   0      0.0      0.0   
         4             0.0  10.2 ...      0.0  23.8   0   0      0.0      0.0   
         5             0.0   1.2 ...      0.0   2.8   0   0      0.0      0.0   

VEH                    HCV      MCV HCV  
TRIP                   SHT   LD EXT EXT  
PERIOD                  PM   PM  PM  PM  
O_County D_County                        
1        1         16407.0  0.0   0   0  
         2             0.0  0.0   0   0  
         3             0.0  0.6   0   0  
         4             0.0  6.8   0   0  
         5             0.0  0.8   0   0  

[5 rows x 24 columns]

### Stack `c2c_flow` table (wide form → long form)

In [9]:
c2c_flow = c2c_flow.stack([0,1,2]).to_frame(name='volume')

In [10]:
c2c_flow.head()

volume
O_County D_County VEH TRIP PERIOD        
1        1        HCV EXT  AM         0.0
                           MID        0.0
                           OFF        0.0
                           PM         0.0
                      LD   AM         0.0

### Select HCV flows
HCV = Heavy-duty commercial vehicles

In [11]:
hcv_flow = c2c_flow\
            .groupby(['O_County', 'D_County','VEH'])\
            .sum()\
            .xs('HCV', level='VEH', axis=0, drop_level=True)

In [12]:
hcv_flow.head(10)

volume
O_County D_County         
1        1         81881.0
         2             0.0
         3             6.0
         4            68.0
         5             8.0
         6             0.0
         7          6401.0
         8             0.0
         9            16.0
         10          265.0

In [13]:
import os
if not os.path.exists(SCRATCH_FOLDER): os.mkdir(SCRATCH_FOLDER)

hcv_flow.to_csv(os.path.join(SCRATCH_FOLDER, 'HCV_ODVol_2040.csv'))